In [44]:
!pip install contractions
!pip uninstall gensim

!pip install gensim==3.8.3
import gensim
import os
import json
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import brown, reuters, gutenberg
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import contractions
import string
from sklearn.linear_model import LinearRegression
from os import listdir
from os.path import isfile, join

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gensim-3.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gensim/*
Proceed (y/n)? y
  Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 24.2 MB 440 kB/s 


In [2]:
!git clone https://github.com/dharace/COMP8730_Assign03_hirpara3.git

Cloning into 'COMP8730_Assign03_hirpara3'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 108 (delta 21), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (108/108), 20.55 MiB | 4.95 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [3]:
%cd /content/COMP8730_Assign03_hirpara3/data/SimLex-999

/content/COMP8730_Assign03_hirpara3/data/SimLex-999


In [4]:
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('brown')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [5]:
def processWords(word):
    # lower case word
    word = word.lower()
    # fix contractions
    text = contractions.fix(word)
    # remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation)) # removing punctuations
    # replace @
    text = text.replace('@', '')
    text = text.split()
    return text

In [6]:
def processList(inputlist):
  result = []
  for sentence in inputlist:
    sentence = ' '.join(sentence)
    sentence = processWords(sentence)
    result.append(sentence)
  return result

In [7]:
brownWords = processList(brown.sents())
brownWords

[['the',
  'fulton',
  'county',
  'grand',
  'jury',
  'said',
  'friday',
  'an',
  'investigation',
  'of',
  'atlantas',
  'recent',
  'primary',
  'election',
  'produced',
  'no',
  'evidence',
  'that',
  'any',
  'irregularities',
  'took',
  'place'],
 ['the',
  'jury',
  'further',
  'said',
  'in',
  'termend',
  'presentments',
  'that',
  'the',
  'city',
  'executive',
  'committee',
  'which',
  'had',
  'overall',
  'charge',
  'of',
  'the',
  'election',
  'deserves',
  'the',
  'praise',
  'and',
  'thanks',
  'of',
  'the',
  'city',
  'of',
  'atlanta',
  'for',
  'the',
  'manner',
  'in',
  'which',
  'the',
  'election',
  'was',
  'conducted'],
 ['the',
  'septemberoctober',
  'term',
  'jury',
  'had',
  'been',
  'charged',
  'by',
  'fulton',
  'superior',
  'court',
  'judge',
  'durwood',
  'pye',
  'to',
  'investigate',
  'reports',
  'of',
  'possible',
  'irregularities',
  'in',
  'the',
  'hardfought',
  'primary',
  'which',
  'was',
  'won',
  'by'

In [8]:
gutenbergWords = processList(gutenberg.sents('austen-emma.txt'))
gutenbergWords

[['emma', 'by', 'jane', 'austen', '1816'],
 ['volume', 'i'],
 ['chapter', 'i'],
 ['emma',
  'woodhouse',
  'handsome',
  'clever',
  'and',
  'rich',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  'and',
  'had',
  'lived',
  'nearly',
  'twenty',
  'one',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her'],
 ['she',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  'indulgent',
  'father',
  'and',
  'had',
  'in',
  'consequence',
  'of',
  'her',
  'sister',
  's',
  'marriage',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period'],
 ['her',
  'mother',
  'had',
  'died',
  'too',
  'long',
  'ago',
  'for',
  'her',
  'to',
  'have',
  'more',
  'than',
  'an',
 

In [9]:
window_size = [1, 2, 5, 10]
vector_size = [10, 50, 100, 300]

In [10]:
def buildWord2VecModel(words, fileName):
  for ws in window_size:
    for vs in vector_size:
        word2vecModel = gensim.models.Word2Vec(window=ws, size=vs, workers=5)
        word2vecModel.build_vocab(words)
        # epochs = 1000 takes lot of time to run, reducing this to 10 easily works for testing
        word2vecModel.train(words, total_examples=word2vecModel.corpus_count, epochs=10) 
        word2vecModel.save(fileName.format(ws,vs))

In [11]:
%cd /content/COMP8730_Assign03_hirpara3/data/Word2Vec/Brown

/content/COMP8730_Assign03_hirpara3/data/Word2Vec/Brown


In [12]:
for ws in window_size:
    for vs in vector_size:
        word2vecModel = gensim.models.Word2Vec(window=ws, size=vs, workers=5)
        word2vecModel.build_vocab(brownWords)
        # epochs = 1000 takes lot of time to run, reducing this to 10 easily works for testing
        word2vecModel.train(brownWords, total_examples=word2vecModel.corpus_count, epochs=10)
        word2vecModel.save('BrownW2V_WS'+ str(ws) + 'VS' + str(vs).format(ws,vs))

In [13]:
%cd /content/COMP8730_Assign03_hirpara3/data/Word2Vec/Guternberg

/content/COMP8730_Assign03_hirpara3/data/Word2Vec/Guternberg


In [14]:
for ws in window_size:
    for vs in vector_size:
        word2vecModel = gensim.models.Word2Vec(window=ws, size=vs, workers=5)
        word2vecModel.build_vocab(gutenbergWords)
        # epochs = 1000 takes lot of time to run, reducing this to 10 easily works for testing
        word2vecModel.train(gutenbergWords, total_examples=word2vecModel.corpus_count, epochs=10)
        word2vecModel.save('GutenW2V_WS'+ str(ws) + 'VS' + str(vs).format(ws,vs))

In [15]:
%cd /content/COMP8730_Assign03_hirpara3/data/SimLex-999

/content/COMP8730_Assign03_hirpara3/data/SimLex-999


In [16]:
dataFile = pd.read_csv('SimLex.csv')

In [17]:
listOfAllWords = dataFile.word1.tolist()
for i,v in enumerate(dataFile.word2.tolist()):
    listOfAllWords.insert(2*i+1,v)
print(listOfAllWords)

['old', 'new', 'smart', 'intelligent', 'hard', 'difficult', 'happy', 'cheerful', 'hard', 'easy', 'fast', 'rapid', 'happy', 'glad', 'short', 'long', 'stupid', 'dumb', 'weird', 'strange', 'wide', 'narrow', 'bad', 'awful', 'easy', 'difficult', 'bad', 'terrible', 'hard', 'simple', 'smart', 'dumb', 'insane', 'crazy', 'happy', 'mad', 'large', 'huge', 'hard', 'tough', 'new', 'fresh', 'sharp', 'dull', 'quick', 'rapid', 'dumb', 'foolish', 'wonderful', 'terrific', 'strange', 'odd', 'happy', 'angry', 'narrow', 'broad', 'simple', 'easy', 'old', 'fresh', 'apparent', 'obvious', 'inexpensive', 'cheap', 'nice', 'generous', 'weird', 'normal', 'weird', 'odd', 'bad', 'immoral', 'sad', 'funny', 'wonderful', 'great', 'guilty', 'ashamed', 'beautiful', 'wonderful', 'confident', 'sure', 'dumb', 'dense', 'large', 'big', 'nice', 'cruel', 'impatient', 'anxious', 'big', 'broad', 'strong', 'proud', 'unnecessary', 'necessary', 'restless', 'young', 'dumb', 'intelligent', 'bad', 'great', 'difficult', 'simple', 'neces

In [18]:
listOfSimLexWords = list(set(listOfAllWords))
print(listOfSimLexWords)

['cheerful', 'replace', 'tear', 'page', 'cigarette', 'hawk', 'argue', 'odd', 'tin', 'west', 'movie', 'misery', 'lend', 'oak', 'effort', 'suds', 'logic', 'carry', 'allow', 'pie', 'cab', 'disease', 'noise', 'hotel', 'order', 'seashore', 'palm', 'stud', 'anatomy', 'student', 'meet', 'attention', 'strength', 'fun', 'game', 'sky', 'generous', 'violet', 'blanket', 'appliance', 'teeth', 'zone', 'take', 'tree', 'state', 'house', 'compare', 'apartment', 'river', 'assignment', 'breakfast', 'happy', 'haze', 'mouth', 'competence', 'pray', 'remind', 'investigate', 'choose', 'cocktail', 'obvious', 'brandy', 'temper', 'anxious', 'spoon', 'navy', 'victory', 'pearl', 'hound', 'recommend', 'ball', 'class', 'engagement', 'build', 'alcohol', 'plane', 'hero', 'winter', 'century', 'crazy', 'legion', 'salt', 'trial', 'scientist', 'happen', 'restless', 'tail', 'fast', 'originate', 'imitate', 'gate', 'hang', 'author', 'bicycle', 'owe', 'elbow', 'cattle', 'chocolate', 'metal', 'floor', 'important', 'sure', 'mel

In [21]:
%cd /content/COMP8730_Assign03_hirpara3/data/SimLex-999

/content/COMP8730_Assign03_hirpara3/data/SimLex-999


In [22]:
with open('SimLex.json', 'r') as f:
  data = json.load(f)
print(data)

words = []
for k,v in data.items():
    words.append(v)
print(words)

{'old': {'new': 7, 'fresh': 9, 'ancient': 9, 'recent': 9, 'wide': 8}, 'new': {'old': 8, 'fresh': 9, 'ancient': 10, 'recent': 10, 'wide': 8, 'modern': 7}, 'smart': {'intelligent': 8, 'dumb': 9, 'stupid': 8, 'foolish': 8, 'dense': 8, 'clever': 8}, 'intelligent': {'smart': 8, 'dumb': 9, 'stupid': 8, 'foolish': 8, 'dense': 8, 'clever': 8}, 'hard': {'difficult': 8, 'easy': 9, 'simple': 9, 'tough': 10, 'dense': 10}, 'difficult': {'hard': 8, 'easy': 9, 'simple': 9, 'tough': 9, 'dense': 9}, 'happy': {'cheerful': 10, 'glad': 10, 'mad': 8, 'angry': 10, 'young': 10, 'delightful': 9}, 'cheerful': {'happy': 7, 'delightful': 10, 'glad': 9, 'mad': 9, 'angry': 9, 'young': 9, 'wonderful': 8}, 'easy': {'hard': 10, 'difficult': 9, 'simple': 9, 'tough': 9, 'flexible': 10, 'big': 10}, 'fast': {'rapid': 4, 'quick': 5}, 'rapid': {'fast': 6, 'quick': 6}, 'glad': {'happy': 8, 'mad': 9, 'cheerful': 9, 'angry': 9, 'young': 9, 'unhappy': 8}, 'short': {'long': 7, 'narrow': 6, 'wide': 7, 'broad': 7, 'dirty': 7}, 'l

In [23]:
%cd /content/COMP8730_Assign03_hirpara3/data/Word2Vec

/content/COMP8730_Assign03_hirpara3/data/Word2Vec


In [24]:
gutenFiles = [f for f in listdir('Guternberg') if isfile(join('Guternberg', f))]
gutenFiles

['GutenW2V_WS2VS50',
 'GutenW2V_WS10VS300',
 'GutenW2V_WS10VS50',
 'GutenW2V_WS1VS100',
 'GutenW2V_WS1VS10',
 'GutenW2V_WS10VS100',
 'GutenW2V_WS5VS10',
 'GutenW2V_WS1VS50',
 'GutenW2V_WS2VS100',
 'GutenW2V_WS5VS300',
 'GutenW2V_WS5VS50',
 'GutenW2V_WS2VS300',
 'GutenW2V_WS2VS10',
 'GutenW2V_WS5VS100',
 'GutenW2V_WS10VS10',
 'GutenW2V_WS1VS300']

In [25]:
brownFiles = [f for f in listdir('Brown') if isfile(join('Brown', f))]
brownFiles

['BrownW2V_WS2VS10',
 'BrownW2V_WS5VS300',
 'BrownW2V_WS2VS100',
 'BrownW2V_WS1VS100',
 'BrownW2V_WS5VS50',
 'BrownW2V_WS5VS10',
 'BrownW2V_WS2VS50',
 'BrownW2V_WS5VS100',
 'BrownW2V_WS10VS10',
 'BrownW2V_WS1VS300',
 'BrownW2V_WS1VS50',
 'BrownW2V_WS10VS50',
 'BrownW2V_WS2VS300',
 'BrownW2V_WS10VS100',
 'BrownW2V_WS10VS300',
 'BrownW2V_WS1VS10']

In [41]:
def getData(files, fName):
  for format in files:
      vect = Word2Vec.load(f)
      result = {}
      for i, w in enumerate(listOfSimLexWords):
          try: 
              x = vect.wv.most_similar(w, 10)
          except:
              x = []
          final_words = [m[0] for m in x]
          temp = {}
          for word in words[i]:
              if word in final_words:
                  id = final_words.index(word)
                  if id in range(0,3):
                      id2rank = 5
                  elif id in range(3,7):
                      id2rank = 4
                  elif id in range(7,10):
                      id2rank = 3
                  temp[word] = id2rank
              else:
                  temp[word] = 0
          result[w] = temp
              
      with open(fName.format(f), 'w') as fp:
          json.dump(result, fp, indent=4)

In [42]:
%cd /content/COMP8730_Assign03_hirpara3/data/Word2Vec/Guternberg

/content/COMP8730_Assign03_hirpara3/data/Word2Vec/Guternberg


In [28]:
!ls

GutenW2V_WS10VS10   GutenW2V_WS1VS10   GutenW2V_WS2VS10   GutenW2V_WS5VS10
GutenW2V_WS10VS100  GutenW2V_WS1VS100  GutenW2V_WS2VS100  GutenW2V_WS5VS100
GutenW2V_WS10VS300  GutenW2V_WS1VS300  GutenW2V_WS2VS300  GutenW2V_WS5VS300
GutenW2V_WS10VS50   GutenW2V_WS1VS50   GutenW2V_WS2VS50   GutenW2V_WS5VS50


In [ ]:
getData(gutenFiles, 'Guternbeg.json')

In [ ]:
%cd /content/COMP8730_Assign03_hirpara3/data/Word2Vec/Brown

In [48]:
getData(brownFiles, 'Brown.json')

AttributeError: ignored